In [179]:
import sqlalchemy as sa
import pandas as pd

In [180]:
from scripts import engine

meta_data = sa.MetaData(bind=engine)
sa.MetaData.reflect(meta_data)

In [181]:
Player=meta_data.tables['Player']
TeamReplayRollup=meta_data.tables['TeamReplayRollup']
Hero=meta_data.tables['Hero']
Battleground=meta_data.tables['Map']
ReplayDetail=meta_data.tables['ReplayDetail']
ReplayPlayer=meta_data.tables['ReplayPlayer']
TeamHeroRollup=meta_data.tables['TeamHeroRollup']
TeamRollup=meta_data.tables['TeamRollup']

In [182]:
team_names = [
    'gold dragons'
    , '30 seconds to mosh'
]

core_cols = [
    'Battletag'
    , 'TeamName'
    , 'MapName'
    , 'GameType'
    , 'GameDate'
    , 'HeroName'
    , 'NewRole'
    , 'Team'
    , 'Party'
    , 'ReplayFingerPrint'
]

group_cols = [
    'Battletag'
    , 'TeamName'
    # , 'MapName'
    # , 'GameType'
    , 'HeroName'
    , 'NewRole'
]

predictors = [
    'GameLengthTimestamp'
    , 'IsWinner'
    , 'Kills'
    , 'Assists'
    , 'Deaths'
    , 'ExperienceContribution'
    , 'FirstToTen'
    , 'Level'
    , 'HeroDamage'
    , 'Healing'
    , 'Takedowns'
    , 'SelfHealing'
    , 'DamageTaken'
    , 'TimeSpentDead'
]

In [183]:
player_replay_df = pd.read_sql(sql = sa.select(TeamReplayRollup), con = engine)
hero_rollup = pd.read_sql(sql = sa.select(TeamHeroRollup), con = engine)
team_rollup = pd.read_sql(sql = sa.select(TeamRollup), con = engine)
team_rollup.to_csv('../sample_data/TeamRollup.csv')

In [184]:
j = ReplayDetail.join(Battleground, Battleground.c.MapId == ReplayDetail.c.MapId) \
    .join(TeamReplayRollup, TeamReplayRollup.c.ReplayFingerPrint == ReplayDetail.c.ReplayFingerPrint)

query = sa.select(sa.distinct(ReplayDetail.c.ReplayFingerPrint)).select_from(j)
replay_detail_df = pd.read_sql(sql = query, con = engine)
replay_detail_df.shape

(28533, 1)

In [185]:
j = ReplayDetail.join(ReplayPlayer, ReplayPlayer.c.ReplayId == ReplayDetail.c.ReplayId) \
    .join(Player, Player.c.PlayerId == ReplayPlayer.c.PlayerId) \
    .join(Battleground, Battleground.c.MapId == ReplayDetail.c.MapId) \
    .join(Hero, Hero.c.HeroId == ReplayPlayer.c.HeroId) \
    .join(TeamReplayRollup, TeamReplayRollup.c.ReplayFingerPrint == ReplayDetail.c.ReplayFingerPrint)

query = sa.select(
    ReplayDetail.c.ReplayFingerPrint
    , TeamReplayRollup.c.TeamName
    , Player.c.Battletag
    , Battleground.c.MapName
    , ReplayDetail.c.GameType
    , Hero.c.HeroName
    , Hero.c.NewRole
    , ReplayDetail.c.GameDate
    , ReplayDetail.c.GameLengthTimestamp
    , ReplayPlayer
    , Player.c.BlizzardId
    , Player.c.MasterPlayerId
).select_from(j)

replay_player_detail_df = pd.read_sql(sql = query, con = engine)
replay_player_detail_df.drop_duplicates(inplace=True)

In [186]:
players = pd.read_sql(sql=sa.select(Player), con=engine)
child_players = players[(players.MasterPlayerId != players.PlayerId)]
child_players.head(100)

,PlayerId,ShortBattletag,Battletag,AccountLevel,BattletagUriEscape,HeroesProfileUrl,Region,BlizzardId,MasterPlayerId
2,3,Aly,Aly#11717,366,Aly%2311717,https://www.heroesprofile.com/Profile/?blizz_i...,1,10958259,1
68,69,NotMebus,NotMebus#1314,100,NotMebus%231314,https://www.heroesprofile.com/Profile/?blizz_i...,1,11101128,4
4454,6657,DadLawnRangr,DadLawnRangr#1834,992,DadLawnRangr%231834,https://www.heroesprofile.com/Profile/?blizz_i...,1,4293850,15372
36465,42892,YipYip,YipYip#11141,258,YipYip%2311141,https://www.heroesprofile.com/Profile/?blizz_i...,1,10807209,4
53688,60830,YapYap,YapYap#11802,199,YapYap%2311802,https://www.heroesprofile.com/Profile/?blizz_i...,1,11063760,97


In [187]:
rc_df = player_replay_df[['ReplayFingerPrint', 'Battletag', 'TeamName', 'Team']].sort_values(['ReplayFingerPrint', 'Team'], ascending=[True, True]).groupby(['ReplayFingerPrint', 'TeamName', 'Team']).agg('count')
rc_df.reset_index(inplace=True)
# rc_df = rc_df[rc_df.TeamName.isin(team_names)]
rc_df = rc_df[rc_df['Battletag'] >= 5]
rc_df.to_sql('TeamReplayTotal', con=engine, if_exists='replace', index=False)
rc_df.to_csv('../sample_data/TeamReplayTotal.csv')

In [188]:
team_replay_df = pd.merge(replay_player_detail_df, rc_df[['ReplayFingerPrint', 'TeamName', 'Team']], left_on=['ReplayFingerPrint', 'TeamName', 'Team'], right_on=['ReplayFingerPrint', 'TeamName', 'Team'])

In [189]:
team_map_df = team_replay_df[['TeamName', 'ReplayFingerPrint', 'MapName', 'IsWinner']].drop_duplicates()
team_map_rollup = team_map_df.groupby(['TeamName', 'MapName']).agg(
    Games=('MapName', 'count')
    , Wins=('IsWinner', 'sum')
)

team_map_rollup['WinRate'] = team_map_rollup['Wins'] / team_map_rollup['Games']
team_map_rollup['WinRate'] = team_map_rollup['WinRate'].round(2)
team_map_rollup.reset_index(inplace=True)
team_map_rollup.to_sql('TeamMapTotal', con=engine, if_exists='replace', index=False)
team_map_rollup.to_csv('../sample_data/TeamMapTotal.csv')

In [190]:
team_hero_df = team_replay_df[['TeamName', 'ReplayFingerPrint', 'HeroName', 'NewRole', 'MapName' ,'IsWinner', 'Battletag']].drop_duplicates()
team_hero_rollup = team_hero_df.groupby(['TeamName', 'HeroName', 'NewRole']).agg(
    Games=('HeroName', 'count')
    , Wins=('IsWinner', 'sum')
)

team_hero_rollup['WinRate'] = team_hero_rollup['Wins'] / team_hero_rollup['Games']
team_hero_rollup['WinRate'] = team_hero_rollup['WinRate'].round(2)
team_hero_rollup.reset_index(inplace=True)
team_hero_rollup.to_sql('TeamHeroTotal', con=engine, if_exists='replace', index=False)
team_hero_rollup.to_csv('../sample_data/TeamHeroTotal.csv')

In [191]:
dfa = team_replay_df.sort_values(['ReplayFingerPrint', 'HeroName', 'Team', 'TeamName'], ascending=[False, True, False, False]).groupby(['ReplayFingerPrint', 'Team', 'TeamName'])['HeroName'].apply(list).to_frame()
dfb = team_replay_df.sort_values(['ReplayFingerPrint', 'NewRole', 'Team', 'TeamName'], ascending=[False, True, False, False]).groupby(['ReplayFingerPrint', 'Team', 'TeamName'])['NewRole'].apply(list).to_frame()
dfc = team_replay_df.sort_values(['ReplayFingerPrint', 'Battletag', 'Team', 'TeamName'], ascending=[False, True, False, False]).groupby(['ReplayFingerPrint', 'Team', 'TeamName'])['Battletag'].apply(list).to_frame()

f_df = dfa.join(dfb).join(dfc)

f_df['Heroes'] = f_df.HeroName.astype(str)
f_df['Roles'] = f_df.NewRole.astype(str)
f_df['Players'] = f_df.Battletag.astype(str)

f_df.reset_index(inplace=True)

f_df.drop('HeroName', inplace=True, axis=1)
f_df.drop('NewRole', inplace=True, axis=1)
f_df.drop('Battletag', inplace=True, axis=1)

f_df = pd.merge(team_replay_df, f_df, left_on=['ReplayFingerPrint', 'Team', 'TeamName'], right_on=['ReplayFingerPrint', 'Team', 'TeamName'])
f_df.to_sql('TeamReplayData', con=engine, if_exists='replace', index=False)
f_df.to_csv('../sample_data/TeamReplayData.csv')

In [192]:
r_rf = team_replay_df[['TeamName', 'ReplayFingerPrint', 'IsWinner']].drop_duplicates()
r_rf = pd.merge(r_rf, f_df[['TeamName', 'ReplayFingerPrint', 'IsWinner', 'GameType']], left_on=['TeamName', 'ReplayFingerPrint', 'IsWinner'], right_on=['TeamName', 'ReplayFingerPrint', 'IsWinner']).drop_duplicates()
r_rf = r_rf.groupby(['TeamName', 'GameType']).agg(
    Games=('GameType', 'count')
    , Wins=('IsWinner', 'sum')
)

r_rf['WinRate'] = r_rf['Wins'] / r_rf['Games']
r_rf['WinRate'] = r_rf['WinRate'].round(2)
r_rf.reset_index(inplace=True)
r_rf.to_sql('TeamGameTypeTotal', con=engine, if_exists='replace', index=False)
r_rf.to_csv('../sample_data/TeamGameTypeTotal.csv')

In [193]:
r_rf = team_replay_df[['TeamName', 'ReplayFingerPrint']].drop_duplicates()
r_rf = pd.merge(r_rf, f_df[['TeamName', 'ReplayFingerPrint', 'GameDate']], left_on=['TeamName', 'ReplayFingerPrint'], right_on=['TeamName', 'ReplayFingerPrint']).drop_duplicates()

r_rf = r_rf.groupby(['TeamName']).agg(
    FirstPlayed=('GameDate', 'min')
    , LastPlayed=('GameDate', 'max')
)
r_rf.reset_index(inplace=True)
r_rf

,TeamName,FirstPlayed,LastPlayed
0,30 seconds to mosh,2021-10-06 01:14:18,2022-08-23 02:55:39
1,beyonce gnolls,2021-12-10 01:27:27,2022-08-18 03:57:26
2,bunker fun time academy,2022-08-16 03:56:27,2022-08-18 02:52:47
3,chogall chocolate factory,2022-07-15 00:59:35,2022-08-17 01:50:59
4,ff & the pancakes,2020-10-02 01:36:34,2022-08-21 01:13:42
5,gold dragons,2022-07-25 02:03:32,2022-08-22 03:45:03
6,infernal misfits,2022-07-08 23:45:05,2022-08-18 01:49:19
7,ltk - we hate obj,2022-07-29 01:37:30,2022-08-17 02:24:00
8,out of pocket,2021-02-09 17:20:35,2022-08-19 02:26:01
9,smoke,2022-08-04 01:37:17,2022-08-18 02:24:15


In [201]:
spike_team_df = hero_rollup #[hero_rollup.TeamName.isin(team_names)]
spike_team_df.head()

,TeamName,TeamRank,Battletag,Rank,BlizzardId,PlayerId,HeroName,NewRole,Total,TotalWins,TotalLosses,HeroWinRate
0,ycmyccastep7:cheeseawakens,G5,shancb9#1737,P2,8220075,1146,Abathur,Support,190,85,105,44.74
1,ycmyccastep7:cheeseawakens,G5,andrade#1700,P5,1160672,186,Illidan,Melee Assassin,4,0,4,0.00
2,ycmyccastep7:cheeseawakens,G5,andrade#1700,P5,1160672,186,Hanzo,Ranged Assassin,2,0,2,0.00
3,ycmyccastep7:cheeseawakens,G5,andrade#1700,P5,1160672,186,Gul'dan,Ranged Assassin,4,1,3,25.00
4,ycmyccastep7:cheeseawakens,G5,andrade#1700,P5,1160672,186,Greymane,Ranged Assassin,14,9,5,64.29


In [202]:
r_df = player_replay_df[[*group_cols, *predictors]].groupby([*group_cols]).agg('mean')
df = pd.merge(spike_team_df, r_df, left_on=['Battletag', 'TeamName', 'HeroName', 'NewRole'], right_on=['Battletag', 'TeamName', 'HeroName', 'NewRole']).sort_values(['TeamName', 'Battletag'])
df_1 = df[df['Total'] >= 5].sort_values(['Battletag', 'NewRole', 'Total'], ascending=[True, True, False]).groupby(['Battletag', 'NewRole']).head(1000)

# df_1['GameLengthTimestamp'] = df_1['GameLengthTimestamp'].round(2)
df_1['IsWinner'] = df_1['IsWinner'].round(2)
df_1['Kills'] = df_1['Kills'].round(2)
df_1['Assists'] = df_1['Assists'].round(2)
df_1['Deaths'] = df_1['Deaths'].round(2)
df_1['ExperienceContribution'] = df_1['ExperienceContribution'].round(2)
df_1['FirstToTen'] = df_1['FirstToTen'].round(2)
df_1['Level'] = df_1['Level'].round(2)
df_1['HeroDamage'] = df_1['HeroDamage'].round(2)
df_1['Healing'] = df_1['Healing'].round(2)
df_1['Takedowns'] = df_1['Takedowns'].round(2)
df_1['SelfHealing'] = df_1['SelfHealing'].round(2)
df_1['DamageTaken'] = df_1['DamageTaken'].round(2)
df_1['TimeSpentDead'] = df_1['TimeSpentDead'].round(2)

df_1

,TeamName,TeamRank,Battletag,Rank,BlizzardId,PlayerId,HeroName,NewRole,Total,TotalWins,...,Deaths,ExperienceContribution,FirstToTen,Level,HeroDamage,Healing,Takedowns,SelfHealing,DamageTaken,TimeSpentDead
2661,30 seconds to mosh,S2,aly#11717,S5,10958259,3,Varian,Bruiser,35,17,...,4.52,9555.45,0.58,20.70,39949.09,0.00,12.21,31637.52,80304.45,182.88
2654,30 seconds to mosh,S2,aly#11717,S5,10958259,3,Stukov,Healer,13,9,...,2.75,9679.50,0.62,21.75,35167.25,56594.00,18.75,37.12,44296.38,100.75
2652,30 seconds to mosh,S2,aly#11717,S5,10958259,3,Rehgar,Healer,9,3,...,5.71,9001.71,0.43,22.29,15160.00,76546.00,12.14,1057.14,69053.86,231.57
2709,30 seconds to mosh,S2,aly#11717,S5,10958259,3,Kharazim,Healer,6,3,...,3.67,8518.33,1.00,21.00,33880.00,28450.67,12.67,0.00,63401.33,154.67
2724,30 seconds to mosh,S2,aly#11717,S5,10958259,3,Ana,Healer,6,2,...,5.00,7502.00,1.00,25.00,31651.00,86584.00,18.00,7944.00,45041.00,264.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,ltk - we hate obj,G5,zoomthenboom#1452,G3,10555838,26768,E.T.C.,Tank,88,37,...,4.74,8387.10,0.41,20.69,23701.92,424.44,11.54,15835.79,72947.10,179.82
1144,ltk - we hate obj,G5,zoomthenboom#1452,G3,10555838,26768,Johanna,Tank,48,31,...,3.44,9971.63,0.44,20.27,40134.15,732.98,13.67,18448.71,81172.92,129.50
1121,ltk - we hate obj,G5,zoomthenboom#1452,G3,10555838,26768,Blaze,Tank,25,14,...,3.34,12190.12,0.53,20.94,44212.16,0.00,13.53,20825.22,91176.78,135.69
1138,ltk - we hate obj,G5,zoomthenboom#1452,G3,10555838,26768,Muradin,Tank,10,5,...,3.56,6874.33,0.56,18.78,25201.78,0.00,9.56,29319.89,85411.22,123.56


In [204]:
df_1.to_sql('TeamMemberHeroTotal', con=engine, if_exists='replace', index=False)
df_1.to_csv('../sample_data/TeamMemberHeroTotal.csv')